# Fine-tune Llama 3 with QLoRA

> 🗣️ [Large Language Model Course](https://github.com/mlabonne/llm-course)

❤️ Created by [@maximelabonne](https://twitter.com/maximelabonne).

You can run this notebook on Google Colab (I use an L4 GPU).

In [1]:
!pip install -qqq -U transformers datasets huggingface_hub accelerate peft bitsandbytes wandb trl --progress-bar off
!FLASH_ATTENTION_SKIP_CUDA_BUILD=TRUE pip install -qqq -U flash-attn --no-build-isolation pip install flash-attn --progress-bar off

In [2]:
from huggingface_hub import login
login()

In [3]:
import gc
import os

import torch
import wandb
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    BitsAndBytesConfig,
    pipeline,
)
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
from trl import SFTTrainer

# Model
base_model = "meta-llama/Meta-Llama-3-8B"
new_model = "Meta-Llama-3-8B-qlora-pos-no-lang"

# Defined in the secrets tab in Google Colab
wb_token = '1d395c70839c926f2dce7fc9403ad88f09e490ba'
wandb.login(key=wb_token)

# Set torch dtype and attention implementation
if torch.cuda.get_device_capability()[0] >= 8:
    !pip install -qqq flash-attn
    torch_dtype = torch.bfloat16
    attn_implementation = "flash_attention_2"
else:
    torch_dtype = torch.float16
    attn_implementation = "eager"

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: kevinxli. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [4]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch_dtype,
    bnb_4bit_use_double_quant=True,
)

# LoRA config
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['up_proj', 'down_proj', 'gate_proj', 'k_proj', 'q_proj', 'v_proj', 'o_proj']
)

pos_tokens = ['pos:']

tokenizer = AutoTokenizer.from_pretrained(base_model)

model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    device_map="auto",
    attn_implementation=attn_implementation,
)
model = prepare_model_for_kbit_training(model)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [5]:
peft_model = get_peft_model(model, peft_config)
print(peft_model.print_trainable_parameters())
print(peft_model)

trainable params: 41,943,040 || all params: 8,072,204,288 || trainable%: 0.5195983464188562
None
PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaFlashAttention2(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterD

In [6]:
from datasets import load_dataset, features

def patch_v(tag):
    if tag == 'V':
        return 'VERB'
    else:
        return tag

def get_dataset(num_existing_tokens=0):
    #default only option
    dataset = load_dataset("hkcancor", "default")

    single_lang = ["eng", "yue", "cmn"]

    tag_name_dict = {}
    for lang in single_lang:
        tag_name_dict[lang] = f'{lang}:'

    source_upos = dataset['train'].features["pos_tags_ud"].feature
    print("Source upos:", source_upos)
    target_upos = features.ClassLabel(
        names=[
            "NOUN",
            "PUNCT",
            "ADP",
            "NUM",
            "SYM",
            "SCONJ",
            "ADJ",
            "PART",
            "DET",
            "CCONJ",
            "PROPN",
            "PRON",
            "X",
            "_",
            "ADV",
            "INTJ",
            "VERB",
            "AUX",
        ]
    )
    print("Target upos:", target_upos)

    def preprocess_function(examples):
        examples["inputs"] = [tag_name_dict["yue"] + ' '.join(examples["tokens"][i]) + "\n" + 
            ''.join(pos_tokens) + ' '.join(patch_v(source_upos.int2str(tag)).lower()
            for tag in examples["pos_tags_ud"][i]) for i in range(len(examples["tokens"]))]
        return examples
    
    dataset['train'] = dataset['train'].map(preprocess_function, remove_columns=
        ['tokens', 'conversation_id', 'pos_tags_prf', 'pos_tags_ud', 'speaker', 'transcriptions', 'turn_number', 'tokens'], batched=True)
    
    dataset = dataset['train'].train_test_split(test_size=0.1, seed=42)
    train_dataset, eval_dataset = dataset['train'], dataset['test']
    return train_dataset, eval_dataset, tag_name_dict

In [7]:
train_dataset, eval_dataset, tag_name_dict = get_dataset()

Source upos: ClassLabel(names=['NUM', 'ADP', 'INTJ', 'PROPN', 'ADJ', 'V', 'DET', 'ADV', 'CCONJ', 'PRON', 'X', 'PART', 'AUX', 'VERB', 'NOUN', 'PUNCT'], id=None)
Target upos: ClassLabel(names=['NOUN', 'PUNCT', 'ADP', 'NUM', 'SYM', 'SCONJ', 'ADJ', 'PART', 'DET', 'CCONJ', 'PROPN', 'PRON', 'X', '_', 'ADV', 'INTJ', 'VERB', 'AUX'], id=None)


In [8]:
train_dataset[:10]

{'inputs': ['yue:噉 考 唔 考 到 啊 ？\npos:cconj verb adv verb part part punct',
  'yue:誒 ， 一 隻 西施 ， 一 隻 拉薩 。\npos:intj punct num noun propn punct num noun propn punct',
  'yue:我 記得 - 記得 嗰陣時 返 羅師 都 係 - 都 係 唔使 一 個 鐘頭 咋 喎 。 即係 開頭 - 係 開頭 返 返 - 即係 最 開頭 嘅 時候 返學 就 要 最 - 最 多 時間 囖 。 成 一 個 鐘頭 零 三 個 字 ， 但係 返 熟 咗 呢 知道 啲 路 點 行 嚹 嗰啲 呢 …\npos:pron verb punct verb pron verb propn adv verb punct adv verb aux num noun noun part part punct cconj adv punct verb adv verb part punct cconj adv adv part noun verb adv verb adv punct adv adj noun part punct adj num noun noun num num noun noun punct cconj verb adj part part verb noun noun pron verb part pron part punct',
  'yue:唔好 啊 。 哩啲 咁 technical ， 冇 興趣 啊 。\npos:aux part punct pron adv adj punct verb noun part punct',
  'yue:諗 清楚 啲 ， 你 讀 語文 你 應該 知 哩個 ， 哩 句 說話 嘅 意思 𡃉 喎 。\npos:verb adj part punct pron verb noun pron aux verb pron punct pron noun noun part noun part part punct',
  'yue:噉 咪 係 囖 。\npos:cconj adv verb part punct',
  'yue:又 係 老細 。\npos:adv verb noun punct

In [9]:
eval_dataset[:10]

{'inputs': ['yue:啲 - 啲 sales 嘅 質素 下降 ， 會 唔 會 好似 同 醫管局 嗰啲 …\npos:noun punct noun noun part noun verb punct aux adv aux verb adp propn pron punct',
  'yue:唉 ， 有 乜嘢 辦法 啊 ， 興 吖 嗎 ， 咁 興 啊 。\npos:intj punct verb pron noun part punct verb part part punct adv verb part punct',
  'yue:夠 嚹 。\npos:verb part punct',
  'yue:噉 另外 就 同 佢 太太 之間 呢 亦都 有 個 中年 婚姻 危機 𡃉 。 即係 覺得 太太 ， 冇 理由 我 太太 會 重 - 重 愛 我 吖 。 即係 我 事業 又 唔 得 ， 要 樣 冇 樣 ， 要 錢 冇 錢 噉樣 。 噉 另外 有 個 黑人 嘅 。 就 係 鬍鬚 阿伯 ， 就 成 五十 歲 嘅 。 點解 會 揾 到 佢 ？ 因為 佢 要 揾 個 人 教 跳舞 吖 嗎 。 而 哩個 阿伯 係 識 跳 所有 爵士舞 ， 只不過 係 因為 骨頭 硬 。 即係 跳 起 身 ， 跌落 地下 之後 就 起 唔 到 身 𡃉 嘞 。 但係 之前 嗰 段 係 得 嘅 。\npos:cconj cconj adv adp pron noun adv part adv verb noun noun noun noun part punct cconj verb noun punct verb noun pron noun aux adv punct adv verb pron part punct cconj pron noun adv adv verb punct aux noun verb noun punct aux noun verb noun cconj punct cconj cconj verb noun noun part punct adv verb noun noun punct adv num num noun part punct pron aux verb part pron punct cconj pron aux verb noun

In [10]:
# Used to suppress:
# Asking to pad but the tokenizer does not have a padding token. Please select a token to use as `pad_token` `(tokenizer.pad_token = tokenizer.eos_token e.g.)` or add a new pad token via `tokenizer.add_special_tokens({'pad_token': '[PAD]'})`.
tokenizer.pad_token = tokenizer.eos_token

In [11]:
args = TrainingArguments(
    learning_rate=5e-05,
    lr_scheduler_type="linear",
    per_device_train_batch_size=12,
    per_device_eval_batch_size=12,
    gradient_accumulation_steps=1,
    optim="paged_adamw_8bit",
    num_train_epochs=1,
    evaluation_strategy="steps",
    eval_steps=0.2,
    logging_steps=1,
    warmup_steps=10,
    report_to="wandb",
    output_dir=f"./results-{new_model}/",
)

trainer = SFTTrainer(
    model=model,
    args=args,
    max_seq_length=512,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    dataset_text_field="inputs",
    tokenizer=tokenizer,
    peft_config=peft_config,
)
trainer.train()
trainer.save_model(new_model)

Map:   0%|          | 0/9720 [00:00<?, ? examples/s]

Map:   0%|          | 0/1081 [00:00<?, ? examples/s]

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
The input hidden states seems to be silently casted in float32, this might be related to the fact you have upcasted embedding or layer norm layers in float32. We will cast back the input in torch.float16.


Step,Training Loss,Validation Loss
162,1.544900,1.650903
324,1.198100,1.528955
486,1.556700,1.483526
648,1.527000,1.446849
810,1.309200,1.431867


/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


In [12]:
trainer.push_to_hub()

adapter_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/5.05k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/AlienKevin/results-Meta-Llama-3-8B-qlora-pos-no-lang/commit/c7c4b1f0ff1aca15af818d37880684252607984e', commit_message='End of training', commit_description='', oid='c7c4b1f0ff1aca15af818d37880684252607984e', pr_url=None, pr_revision=None, pr_num=None)